In [6]:
import skrub
import skrub.datasets

dataset = skrub.datasets.fetch_credit_fraud()
skrub.TableReport(dataset.baskets)

Processing column   2 / 2


<TableReport: use .open() to display>

In [7]:
skrub.TableReport(dataset.products)

Processing column   7 / 7


,,,,,,,


In [8]:
products = skrub.var("products", dataset.products)
baskets = skrub.var("baskets", dataset.baskets)

basket_ids = baskets[["ID"]].skb.mark_as_X()
fraud_flags = baskets["fraud_flag"].skb.mark_as_y()

In [9]:
kept_products = products[products["basket_ID"].isin(basket_ids["ID"])]
products_with_total = kept_products.assign(
    total_price=kept_products["Nbr_of_prod_purchas"] * kept_products["cash_price"]
)
products_with_total

<CallMethod 'assign'>
Result:
―――――――
        basket_ID                              item  cash_price         make  \
1           51113  COMPUTER PERIPHERALS ACCESSORIES         409        APPLE   
9           41798                         COMPUTERS        1187        APPLE   
11          39361                         COMPUTERS         898        APPLE   
15          38615  COMPUTER PERIPHERALS ACCESSORIES         379        APPLE   
16          70262                         COMPUTERS        1899        APPLE   
...           ...                               ...         ...          ...   
163352      42613                 BEDROOM FURNITURE         259  SILENTNIGHT   
163353      42613                 OUTDOOR FURNITURE         949   LG OUTDOOR   
163354      43567                         COMPUTERS        1099        APPLE   
163355      43567                         COMPUTERS        2099        APPLE   
163356      68268           TELEVISIONS HOME CINEMA         799           LG   

                                                    model goods_code  \
1       APPLE WATCH SERIES 6 GPS 44MM SPACE GREY ALUMI...  239001518   
9       2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESS...  239246780   
11      2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 ...  239246776   
15      APPLE WATCH SERIES 6 GPS 40MM BLUE ALUMINIUM CASE  239001540   
16      2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16G...  240575990   
...                                                   ...        ...   
163352  SILENTNIGHT SLEEP GENIUS FULL HEIGHT HEADBOARD...  236938439   
163353  LG OUTDOOR BERGEN 2-SEAT GARDEN SIDE TABLE REC...  239742814   
163354  2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-F...  240040978   
163355  2020 APPLE IMAC 27 ALL-IN-ONE INTEL CORE I7 8G...  238923518   
163356  LG OLED48A16LA 2021 OLED HDR 4K ULTRA HD SMART TV  239866717   

        Nbr_of_prod_purchas  total_price  
1                         1          409  
9                         1         1187  
11                        1          898  
15                        1          379  
16                        1         1899  
...                     ...          ...  
163352                    1          259  
163353                    1          949  
163354                    1         1099  
163355                    1         2099  
163356                    1          799  

[109380 rows x 8 columns]

In [10]:
n = skrub.choose_int(5, 15, name="n_components")
encoder = skrub.choose_from(
    {
        "MinHash": skrub.MinHashEncoder(n_components=n),
        "LSA": skrub.StringEncoder(n_components=n),
    },
    name="encoder",
)
vectorizer = skrub.TableVectorizer(high_cardinality=encoder)

In [11]:
vectorized_products = products_with_total.skb.apply(
    vectorizer, exclude_cols="basket_ID"
)

In [12]:
aggregated_products = vectorized_products.groupby("basket_ID").agg("mean").reset_index()
augmented_baskets = basket_ids.merge(
    aggregated_products, left_on="ID", right_on="basket_ID"
).drop(columns=["ID", "basket_ID"])

In [13]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(
    learning_rate=skrub.choose_float(0.01, 0.9, log=True, name="learning_rate")
)
predictions = augmented_baskets.skb.apply(hgb, y=fraud_flags)
predictions

<Apply HistGradientBoostingClassifier>
Result:
―――――――
0        0
1        0
2        0
3        0
4        0
        ..
61236    0
61237    0
61238    0
61239    0
61240    0
Name: fraud_flag, Length: 61241, dtype: int64